# Common imports & library functions

In [1]:
import doctest
import itertools
import math
from collections import defaultdict

# Day 6: Universal Orbit Map

In [61]:
COM = 'COM'
YOU = 'YOU'
SAN = 'SAN'

def orbit_count_checksum(map):
    """
    >>> orbit_count_checksum('COM)B\\nB)C\\nC)D')
    6
    """
    children = defaultdict(set)
    for edge in map.split():
        p, c = edge.split(')')
        children[p].add(c)
    levels = {COM: 0}
    to_visit = {COM}
    while to_visit:
        p = to_visit.pop()
        for c in children[p]:
            levels[c] = levels[p] + 1
            to_visit.add(c)
    return sum(levels.values())

def num_orbital_transfers(map, source=YOU, target=SAN):
    parents = {COM: None}
    for edge in map.split():
        p, c = edge.split(')')
        parents[c] = p
    def path_to(node):
        if not node:
            return []
        return path_to(parents[node]) + [node]
    source_path = path_to(source)
    target_path = path_to(target)
    in_common = len(list(itertools.takewhile(
        lambda p: p[0] == p[1], zip(source_path, target_path))))
    return len(source_path[in_common:]) + len(target_path[in_common:]) - 2

In [37]:
# Run unit tests
doctest.run_docstring_examples(orbit_count_checksum, globs=None, verbose=True)

Finding tests in NoName
Trying:
    orbit_count_checksum('COM)B\nB)C\nC)D')
Expecting:
    6
ok


In [38]:
# Integration test:
test_map = """
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
"""
assert orbit_count_checksum(test_map) == 42

In [60]:
# Integration test:
test_map = """
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN
"""
assert num_orbital_transfers(test_map) == 4

['D', 'E', 'J', 'K', 'YOU']
['D', 'I', 'SAN']


In [62]:
# Final answers
with open('day6_input.txt') as f:
    map = f.read()
    print('Part 1: ', orbit_count_checksum(map))
    print('Part 2: ', num_orbital_transfers(map))

Part 1:  119831
Part 2:  322


# Day 7: Amplification Circuit

In [46]:
import asyncio
import intcode

import imp
import itertools

In [149]:
imp.reload(intcode)

<module 'intcode' from 'D:\\My Library\\Documents\\GitHub\\advent-of-code-2019\\intcode.py'>

In [75]:
import dataclasses
from typing import List

@dataclasses.dataclass
class Amp:
    mem: List[int]
    iq: asyncio.Queue
    oq: asyncio.Queue
    halted: bool = False

def make_amp(amp_controller_code, inputs, iq=None, oq=None):
    iq = iq or asyncio.Queue()
    oq = oq or asyncio.Queue()
    amp = Amp(amp_controller_code, iq, oq)
    for i in inputs:
        amp.iq.put_nowait(i)
    return amp

async def run_amp(amp):
    while True:
        _, ip = await execute(amp.mem, input=amp.iq.get, output=amp.oq.put_nowait)
        if ip == -1:
            amp.halted = True
            break

async def run_amp_system(amp_controller_code, phase_settings):
    num_amps = len(phase_settings)
    amps = []
    input_queues = [asyncio.Queue() for _ in range(num_amps)]
    output_queues = input_queues[1:] + input_queues[0:1]
    for i, phase in enumerate(phase_settings):
        amps.append(make_amp(amp_controller_code, [phase], iq=input_queues[i], oq=output_queues[i]))
    
    amps[0].iq.put_nowait(0)
    tasks = [asyncio.create_task(run_amp_with_feedback(amp)) for amp in amps]
    await asyncio.gather(*tasks)
    return await amps[0].iq.get()

async def find_best_phase_settings(amp_controller_code, with_feedback=False):
    phase_range = range(5, 10) if with_feedback else range(5)
    phase_settings = itertools.permutations(phase_range)
    outputs = await asyncio.gather(*(run_amp_system(amp_controller_code, phase_setting)
                                     for phase_setting in phase_settings))
    return max(outputs)

In [67]:
# Run unit tests
await run_amp_system_with_feedback([3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0], [4,3,2,1,0]) == 43210
await run_amp_system_with_feedback([3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0], [0,1,2,3,4]) == 54321
await run_amp_system_with_feedback([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5], [9,8,7,6,5]) == 139629729
await run_amp_system_with_feedback([3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
                                    -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
                                    53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10], [9,7,8,5,6]) == 18216    

True

In [77]:
# Final answers
with open('day7_input.txt') as f:
    amp_controller_code = intcode.parse_program(f.read())
    print('Part 1: ', await find_best_phase_settings(amp_controller_code))
    print('Part 2: ', await find_best_phase_settings(amp_controller_code, with_feedback=True))

Part 1:  43812
Part 2:  59597414


# Day 8: Space Image Format

In [114]:
import dataclasses
from typing import List

Layer = str

@dataclasses.dataclass
class Image:
    layers: List[Layer]
    width: int
    height: int

def parse_image(raw_image, width, height):
    """
    >>> parse_image('123456789012', 3, 2)
    Image(layers=['123456', '789012'], width=3, height=2)
    """
    raw_image = raw_image.strip()
    layer_size = width * height
    assert len(raw_image) % layer_size == 0
    return Image(layers=[raw_image[i:i+layer_size]
                         for i in range(0, len(raw_image), layer_size)],
                 width=width,
                 height=height)

def render_layer(layer, width, height):
    """
    >>> render_layer('123456', 3, 2)
    123
    456
    """
    for i in range(0, len(layer), width):
        print(layer[i:i+width])

def render_image(image):
    """
    >>> render_image(parse_image('123456789012', 3, 2))
    Layer 1:
    # 3
    456
    Layer 2:
    789
    .# 
    """
    for i, layer in enumerate(image.layers):
        print(f'Layer {i+1}:')
        layer = layer.replace('0', '.').replace('1', '#').replace('2', ' ')
        render_layer(layer, image.width, image.height)

def first_opaque_pixel(raw_pixels):
    """
    >>> first_opaque_pixel('2221')
    '1'
    >>> first_opaque_pixel('0222')
    '0'
    >>> first_opaque_pixel('222')
    """
    return next(itertools.dropwhile(lambda p: p == '2', raw_pixels), None)
        
def flatten_image(image):
    """
    >>> image = parse_image('0222112222120000', 2, 2)
    >>> flatten_image(image)
    Image(layers=['0110'], width=2, height=2)
    """
    layer = [first_opaque_pixel(pixels)
             for pixels in zip(*image.layers)]
    return Image(layers=[''.join(layer)], width=image.width, height=image.height)
    
def corruption_test_part_1(image):
    layer = min(image.layers, key=lambda l: l.count('0'))
    return layer.count('1') * layer.count('2')

In [115]:
# Run unit tests
doctest.run_docstring_examples(parse_image, globs=None, verbose=True)
doctest.run_docstring_examples(render_layer, globs=None, verbose=True)
doctest.run_docstring_examples(render_image, globs=None, verbose=True)
doctest.run_docstring_examples(first_opaque_pixel, globs=None, verbose=True)
doctest.run_docstring_examples(flatten_image, globs=None, verbose=True)

Finding tests in NoName
Trying:
    parse_image('123456789012', 3, 2)
Expecting:
    Image(layers=['123456', '789012'], width=3, height=2)
ok
Finding tests in NoName
Trying:
    render_layer('123456', 3, 2)
Expecting:
    123
    456
ok
Finding tests in NoName
Trying:
    render_image(parse_image('123456789012', 3, 2))
Expecting:
    Layer 1:
    # 3
    456
    Layer 2:
    789
    .# 
ok
Finding tests in NoName
Trying:
    first_opaque_pixel('2221')
Expecting:
    '1'
ok
Trying:
    first_opaque_pixel('0222')
Expecting:
    '0'
ok
Trying:
    first_opaque_pixel('222')
Expecting nothing
ok
Finding tests in NoName
Trying:
    image = parse_image('0222112222120000', 2, 2)
Expecting nothing
ok
Trying:
    flatten_image(image)
Expecting:
    Image(layers=['0110'], width=2, height=2)
ok


In [113]:
# Final answers
with open('day8_input.txt') as f:
    raw_image = f.read()
    image = parse_image(raw_image, 25, 6)
    print('Part 1: ', corruption_test_part_1(image))
    print('Part 2:')
    flattened = flatten_image(image)
    render_image(flattened)

Part 1:  1441
Part 2:
Layer 1:
###..#..#.####.###..###..
#..#.#..#....#.#..#.#..#.
#..#.#..#...#..###..#..#.
###..#..#..#...#..#.###..
#.#..#..#.#....#..#.#....
#..#..##..####.###..#....
